In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# --- Chargement de la table clients ---
clients_all_df = pd.read_csv(r"C:\Users\MSI\Desktop\Attijari_bank\stage\clients.csv")
clients_all_df = clients_all_df[['id_client', 'type_client', 'salaire', 'ville']]
clients_all_df['salaire'] = pd.to_numeric(clients_all_df['salaire'], errors='coerce').fillna(0)

# --- Variables et listes ---
cartes_pro = [
    'Corporate', 'Gold Professionnelle', 'Visa Business', 'MasterCard Business',
    'Gold Internationale', 'Visa Internationale', 'TAWA TAWA', 'CIB Professionnelle'
]

cartes_particulier = [
    'Visa Nationale', 'Gold Nationale', 'Platinum', 'CIB', 'TAWA TAWA', 'Voyage', 'Iddikhar'
]

account_types_particulier = ['Compte Epargne', 'Compte DAV', 'Compte en Devise', 'Compte Flex']
account_types_pro = ['Compte DAV']
etats_possibles = ['Actif', 'Bloque', 'Ferme']

start_date = datetime(2015, 1, 1)
end_date = datetime(2023, 12, 31)

# --- Fonctions utilitaires ---
def generate_account_id(existing_ids):
    while True:
        acc_id = str(random.randint(1000000000, 9999999999))
        if acc_id not in existing_ids:
            return acc_id

def generate_random_date(start, end):
    delta = end - start
    return start + timedelta(days=random.randint(0, delta.days))

def assign_card(type_client, type_compte):
    if type_client == 'Professionnel':
        return random.choice(cartes_pro)
    else:
        if type_compte == 'Compte DAV':
            return random.choice(['CIB', 'Visa Nationale', 'Gold Nationale', 'TAWA TAWA', 'Platinum'])
        elif type_compte == 'Compte Epargne':
            return 'Iddikhar'
        elif type_compte == 'Compte en Devise':
            return random.choice(['Voyage', 'Visa Internationale'])
        elif type_compte == 'Compte Flex':
            return random.choice(['TAWA TAWA', 'CIB'])

def generate_etat(type_client):
    probs = [0.85, 0.1, 0.05]  # Actif, Bloque, Ferme
    return random.choices(etats_possibles, weights=probs, k=1)[0]

def generate_solde(type_compte, salaire, etat):
    if etat in ['Bloque', 'Ferme']:
        return round(random.uniform(0, 50), 2)
    else:
        if type_compte == 'Compte Epargne':
            return round(random.uniform(200, max(200, salaire * 2.5)), 2)
        elif type_compte == 'Compte DAV':
            return round(random.uniform(100, max(100, salaire * 1.5)), 2)
        elif type_compte == 'Compte en Devise':
            return round(random.uniform(300, max(300, salaire * 2)), 2)
        elif type_compte == 'Compte Flex':
            return round(random.uniform(50, max(50, salaire * 0.5)), 2)
        elif type_compte == 'Compte Professionnel':
            return round(random.uniform(1.0, 2.5) * salaire, 2)
        return 0.0

# --- Génération des comptes ---
accounts = []
existing_ids = set()

for idx, row in clients_all_df.iterrows():
    id_client = row['id_client']
    type_client = row['type_client']
    salaire = row['salaire']
    ville = row['ville']

    nb_comptes = random.randint(1, 3)

    for _ in range(nb_comptes):
        account_id = generate_account_id(existing_ids)
        existing_ids.add(account_id)

        if type_client == 'Etudiant':
            type_compte = 'Compte Epargne'
        elif type_client == 'Chomeur':
            autres = [acc for acc in account_types_particulier if acc != 'Compte Professionnel']
            type_compte = random.choice(autres)
        elif type_client == 'Professionnel':
            type_compte = 'Compte Professionnel'
        else:
            if random.random() < 0.75:
                type_compte = 'Compte DAV'
            else:
                autres = [acc for acc in account_types_particulier if acc != 'Compte DAV']
                type_compte = random.choice(autres)

        etat = generate_etat(type_client)
        solde = generate_solde(type_compte, salaire, etat)

        if etat == 'Actif' and solde <= 0:
            solde = round(random.uniform(50, max(50, salaire * 1.2)), 2)

        carte = assign_card(type_client, type_compte)
        date_ouverture = generate_random_date(start_date, end_date).date()

        if etat == 'Actif':
            if type_client in ['Etudiant', 'Chomeur']:
                eligible_chequier = False
            elif type_client == 'Professionnel':
                eligible_chequier = True
            elif type_client == 'Particulier' and type_compte == 'Compte DAV':
                eligible_chequier = True
            elif type_client in ['Entreprise', 'VIP']:
                eligible_chequier = True
            else:
                eligible_chequier = False
        else:
            eligible_chequier = False

        account = {
            'id_compte': account_id,
            'id_client': id_client,
            'type_client': type_client,
            'type_compte': type_compte,
            'type_carte': carte,
            'date_ouverture': date_ouverture,
            'solde_initial': solde,
            'etat_compte': etat,
            'eligible_chequier': eligible_chequier,
            'agence': f"Attijari Bank {ville}"
        }

        accounts.append(account)

# --- Conversion en DataFrame ---
compte_df = pd.DataFrame(accounts)

# --- Ajout de colonnes : deja_cheque et demande_cheque ---
compte_df['deja_cheque'] = False
compte_df['demande_cheque'] = False

eligibles_idx = compte_df[compte_df['eligible_chequier'] == True].index
nb_deja_cheque = int(0.75 * len(eligibles_idx))
idx_deja_cheque = np.random.choice(eligibles_idx, size=nb_deja_cheque, replace=False)
compte_df.loc[idx_deja_cheque, 'deja_cheque'] = True

reste_eligibles = eligibles_idx.difference(idx_deja_cheque)
nb_demande_cheque = int(0.60 * len(reste_eligibles))
idx_demande_cheque = np.random.choice(reste_eligibles, size=nb_demande_cheque, replace=False)
compte_df.loc[idx_demande_cheque, 'demande_cheque'] = True

# --- Vérification finale ---
nb_clients_total = clients_all_df['id_client'].nunique()
nb_comptes_total = compte_df.shape[0]
nb_clients_dans_comptes = compte_df['id_client'].nunique()

print(f"👥 Nombre total de clients          : {nb_clients_total}")
print(f"🏦 Nombre total de comptes         : {nb_comptes_total}")
print(f"🔗 Clients représentés dans comptes: {nb_clients_dans_comptes}")

if nb_clients_total == nb_clients_dans_comptes:
    print("✅ Tous les clients ont au moins un compte.")
else:
    print("❌ Certains clients n'ont pas de compte.")


👥 Nombre total de clients          : 180000
🏦 Nombre total de comptes         : 359723
🔗 Clients représentés dans comptes: 180000
✅ Tous les clients ont au moins un compte.


In [2]:
# --- Vérification finale ---
nb_clients_total = clients_all_df['id_client'].nunique()
nb_comptes_total = compte_df.shape[0]
nb_clients_dans_comptes = compte_df['id_client'].nunique()

print(f"👥 Nombre total de clients          : {nb_clients_total}")
print(f"🏦 Nombre total de comptes         : {nb_comptes_total}")
print(f"🔗 Clients représentés dans comptes: {nb_clients_dans_comptes}")

if nb_clients_total == nb_clients_dans_comptes:
    print("✅ Tous les clients ont au moins un compte.")
else:
    print("❌ Certains clients n'ont pas de compte.")

# --- Suppression de la colonne type_client ---
compte_df = compte_df.drop(columns=['type_client'])

# --- Export CSV (optionnel) ---
chemin_fichier = r"C:\Users\MSI\Desktop\Attijari_bank\stage\comptes.csv"
compte_df.to_csv(chemin_fichier, index=False)
print(f"✅ Fichier comptes.csv généré avec {len(compte_df)} comptes.")


👥 Nombre total de clients          : 180000
🏦 Nombre total de comptes         : 359723
🔗 Clients représentés dans comptes: 180000
✅ Tous les clients ont au moins un compte.
✅ Fichier comptes.csv généré avec 359723 comptes.


In [5]:
compte_df.head(10)

,id_compte,id_client,type_compte,type_carte,date_ouverture,solde_initial,etat_compte,eligible_chequier,agence,deja_cheque,demande_cheque
0,9070725833,1702993,Compte Professionnel,Corporate,2020-03-12,8706.10,Actif,True,Attijari Bank Sidi Bouzid,True,False
1,1956728031,1702993,Compte Professionnel,CIB Professionnelle,2018-02-07,7627.96,Actif,True,Attijari Bank Sidi Bouzid,False,True
2,1347916419,4762995,Compte Professionnel,Visa Business,2016-06-06,13321.57,Actif,True,Attijari Bank Mednine,False,False
3,2845670947,4762995,Compte Professionnel,Visa Business,2022-09-22,13.34,Ferme,False,Attijari Bank Mednine,False,False
4,9014185170,4762995,Compte Professionnel,Corporate,2022-10-08,15447.26,Actif,True,Attijari Bank Mednine,True,False
5,3595729910,4234274,Compte DAV,CIB,2020-10-29,781.58,Actif,True,Attijari Bank Sidi Bouzid,True,False
6,3421906866,4234274,Compte DAV,Platinum,2018-04-30,44.71,Bloque,False,Attijari Bank Sidi Bouzid,False,False
7,8118696792,4892162,Compte Professionnel,Visa Internationale,2019-12-13,10.49,Bloque,False,Attijari Bank La Marsa,False,False
8,3942698849,4892162,Compte Professionnel,TAWA TAWA,2018-08-23,12429.38,Actif,True,Attijari Bank La Marsa,True,False
9,2419622709,4747424,Compte DAV,Gold Nationale,2019-07-21,35.21,Bloque,False,Attijari Bank La Manouba,False,False
